In [56]:
import requests
from iso3166 import countries
from commondata_languages import LanguageData

#=== LISTS AND DICTIONARIES OF COUNTRY AND LANGUAGE CODES ===
country_dict = {country.name.title(): country.alpha2.lower() for country in countries}
country_list = sorted([country.name.title() for country in countries])
language_dict = {language.name: language.iso1 for language in LanguageData() if language.iso1 is not None}
language_list = sorted([language.name for language in LanguageData() if language.iso1 is not None])

#=== API PARAMETERS ===
API_KEY = "0223db48f9b941f3a13b6ffa842607aa"
BASE_URL = "https://api.worldnewsapi.com/search-news"

#=== API CALLS ===
def get_search_news_results(query: str, language: str | None = None, country: str | None = None, offset: str = "0"):
    params = {
        "api-key" : API_KEY,
        "text" : f"\"{query.strip()}\"",
        "text-match-indexes" : "title,content",
        "number": "5",
        "offset": offset
    }

    if language is not None:
        params["language"] = language.lower()

    if country is not None:
        params["source-country"] = country.lower()


    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

#=== SEARCH FUNCTION
def search_news():
    choice = input("Q=Quick Search, S=Specific Search: ")
    if choice.lower() == "q":
        query = input("Type in your query: ")
        data = get_search_news_results(query)
        print_results(data)
    elif choice.lower() == "s":
        query = input("Type in your query: ")

        country_code = None
        lang_code = None

        isCountry = False
        while not isCountry:
            country = input("Country: ")
            if country.title() in country_list:
                country_code = country_dict[country.title()]
                isCountry = True
            else:
                print(f"\"{country}\" is not in our database.")
                print("Please recheck the spelling or try its other names.")
        
        isLanguage = False
        while not isLanguage:
            search_language = input("Language: ")
            language = LanguageData()[search_language]
            if language is not None:
                if search_language.lower() != language.name.lower():
                    print(f"Searching using {language.name.title()} instead...")
                lang_code = language_dict[language.name]
                isLanguage = True
            else:
                print(f"\"{search_language}\" is not in our database.")
                print("Please recheck the spelling or try its other names.")
        
        data = get_search_news_results(query, lang_code, country_code)
        print_results(data)

def print_results(data):
    print("===== Search Results =====")
    print(f"There are {data["available"]} articles with your query.")
    for entry in data["news"]:
        print(f"\nTitle: {entry["title"]}")
        print(f"Author: {entry["author"]}")
        print(f"Date Published: {entry["publish_date"]}")
        print(f"Language: {next((language for language, code in language_dict.items() if code == entry["language"].lower()), None)} ||", end=" ")
        print(f"Country: {next((country for country, code in country_dict.items() if code == entry["source_country"].lower()), None)}")
        print(f"Text = {entry["text"][:297].replace('\n', ' ') + "..."}")
        print(f"Continue reading at: {entry["url"]}")

def main():
    print("===== News Article Search =====")
    print("\n")
    search_news()

if __name__ == "__main__":
    main()


===== News Article Search =====


===== Search Results =====
There are 424 articles with your query.

Title: Insulating the military
Author: Edgard A. Arevalo
Date Published: 2025-11-22 16:08:00
Language: English || Country: Philippines
Text = PERSISTENT public calls for transparency in investigations on the corruption of public funds for flood control projects are resonating loudly. The frustration over these is not just due to the apparent lackadaisical filing of cases by the Ombudsman, until recently. The clamor is to see that gover...
Continue reading at: https://www.manilatimes.net/2025/11/23/opinion/columns/insulating-the-military/2229780

Title: BARMM districting law unlikely to be passed this year
Author: Ali Macabalang
Date Published: 2025-11-22 16:10:00
Language: English || Country: Philippines
Text = COTABATO CITY — The deputy speaker of the Bangsamoro Transition Authority (BTA) Parliament said a new Bangsamoro Autonomous Region in Muslim Mindanao (BARMM) districting law may